In [22]:
from pybaseball import statcast
from pybaseball import playerid_reverse_lookup
import matplotlib.pylab as plt
import pandas as pd
import seaborn as sb

#2018 Data

# Speed Calculation#

In [282]:
spd = pd.read_csv("sprint_speed.csv")
acc = pd.read_csv("running_splits.csv")

acc=acc[['last_name', 'player_id', 'seconds_since_hit_010', 'seconds_since_hit_030']]
#acc.sort_values(by='seconds_since_hit_010', ascending=True).head()

spd=spd[['last_name', 'player_id', 'sprint_speed']]
#spd.sort_values(by='sprint_speed', ascending=False).head()

acc=acc.merge(spd, how="left", on='player_id')
spd=acc[['last_name_x', 'player_id', 'seconds_since_hit_010', 'seconds_since_hit_030', 'sprint_speed']]
spd=spd.rename(index=str, columns={'last_name_x':'last_name'})

spd['SPD']=(50-spd['seconds_since_hit_010']*30)+(30-spd['seconds_since_hit_030']*10)+spd['sprint_speed']*2
spd.drop_duplicates(subset='player_id', keep='first', inplace=True)
spd=spd[['last_name', 'player_id', 'SPD']].sort_values(by='SPD', ascending=False)
spd.head(10)

,last_name,player_id,SPD
175,Hamilton,571740,99.8
120,Engel,641553,99.4
367,Sierra,642423,99.3
372,Smith,605480,99.1
99,DeShields,592261,99.1
117,Dyson,502481,98.5
1,Acuna Jr.,660670,98.4
133,Fowler,641583,98.2
401,Turner,607208,97.8
27,Bader,664056,97.8


# Strength Calculation #
fb/ld is fly ball or line drive so it assumes the ball is well hit
95.8 is minimum max hit speed
ev95 is % of hits that are hard hit (over 95mph) -10.8 to scale to 100

In [233]:
batting = pd.read_csv("exit_velocity.csv")

batting=batting[['last_name', 'player_id', 'max_hit_speed', 'fbld', 'ev95percent']]
batting['STR']=(batting['max_hit_speed']-95.8)+(batting['fbld']/3)+(batting['ev95percent']-10.8)
batting=batting[['last_name', 'player_id', 'STR']].sort_values(by='STR', ascending=False)
batting.head(10)

,last_name,player_id,STR
2,Judge,592450,99.966667
0,Stanton,519317,98.833333
45,Cabrera,408234,95.100000
14,Martinez,502110,94.633333
11,Cruz,443558,94.100000
3,Palka,594953,93.466667
6,Gallo,608336,93.300000
55,Bote,623520,92.400000
12,Cordero,614173,92.033333
111,Voit,572228,91.866667


# Play Making Ability Calculation #

In [232]:
fielding = pd.read_csv("catch_probability.csv")
pop=pd.read_csv("poptime.csv")

fielding=fielding[['last_name', 'player_id', 'n_5star_percent', 'n_4star_percent', 'n_3star_percent']]

fielding['ABL']=fielding['n_5star_percent']+(fielding['n_4star_percent']/2)+(fielding['n_3star_percent']/3)
fielding=fielding[['last_name', 'player_id', 'ABL']].sort_values(by='ABL', ascending=False)
#fielding.head()

pop=pop[['catcher', 'player_id', 'maxeff_arm_2b_3b_sba', 'pop_2b_sba']]
pop['ABL']=(pop['maxeff_arm_2b_3b_sba']/2)+(50-(pop['pop_2b_sba']*20))
pop=pop[['player_id', 'ABL']].sort_values(by='ABL', ascending=False)
#pop.head()

fielding=fielding.append(pop, sort=False)
fielding=fielding.sort_values(by='ABL', ascending=False)
fielding.drop_duplicates(subset='player_id', keep='first', inplace=True)
fielding.head(10)

,last_name,player_id,ABL
0,Cain,456715,99.616667
2,Bader,664056,99.533333
21,Bellinger,641355,87.933333
1,Inciarte,542255,80.483333
3,Hamilton,571740,80.300000
5,Margot,622534,76.066667
7,DeShields,592261,74.250000
15,Reddick,502210,71.983333
17,Kiermaier,595281,70.316667
12,Taylor,572191,70.016667


In [230]:
ath = fielding.merge(batting, how='left', on='player_id').merge(spd, how="left", on='player_id')
ath = ath[['last_name', 'player_id', 'ABL', 'STR', 'SPD']]
ath['ATH']=(ath['ABL']/6 + ath['STR']/2 + ath['SPD']/2)-4
ath=ath.sort_values(by='ATH', ascending=False)
ath.head(10)

,last_name,player_id,ABL,STR,SPD,ATH
26,Trout,545361,58.133333,89.566667,97.5,99.222222
12,Acuna Jr.,660670,66.650000,85.766667,98.4,99.191667
131,Judge,592450,31.450000,99.966667,93.9,98.175000
0,Cain,456715,99.616667,76.633333,94.0,97.919444
15,Bradley Jr.,598265,64.116667,88.033333,94.4,97.902778
13,Betts,605141,65.916667,85.933333,94.2,97.052778
2,Bellinger,641355,87.933333,76.200000,95.8,96.655556
120,Gallo,608336,40.433333,93.300000,92.8,95.788889
1,Bader,664056,99.533333,68.333333,97.8,95.655556
17,Grichuk,545341,62.983333,82.533333,94.9,95.213889


# Statistic Evaluation #
<p> There are no statistics looking directly at player athleticism available to the public. Because of this, I will compare my stat to Baseball References WAR statistic. WAR shows the statistical production of players, which is essentially a product of baseball skills and athleticism. Because my ATH stat shows player athleticism, it should correlate well with fWAR. </p>

In [280]:
from scipy import stats
from pybaseball import bwar_bat
bWAR=bwar_bat()
bWAR=bWAR[bWAR['year_ID']==2018]
bWAR=bWAR[['mlb_ID', 'WAR']]
bWAR['player_id']=bWAR[['mlb_ID']]
bWAR=bWAR[['player_id', 'WAR']]
ath=ath.merge(bWAR, how='left', on='player_id')

In [283]:
import numpy as np

ath=ath.dropna(axis=0)
np.corrcoef(ath['ATH'], ath['WAR'])

array([[1.        , 0.45967685],
       [0.45967685, 1.        ]])

<p> The correlation coefficient between my ATH stat and Baseball Reference's WAR is 0.459, which, like I wrote above, shows that they are partially correlated. </p>